TRABALHO BD

A partir de webscrapping, ir buscar o número de ids escolhido (máx 20) de termos à escolha do utilizador e importar as informações para a base de dados

WEBSCRAPPING

In [12]:
import time
query = input("Defina o que pretende pesquisar [bacteria/yeast/...]: ")
def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(query)
    url_list.append(url)
    return url_list
    
url_get(0)

['https://www.ncbi.nlm.nih.gov/gene/?term=canis']

In [14]:
import requests, sys, json
content = []
for url in url_get(0):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [15]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [21]:
import re
existe = re.findall(r'\s*ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'\s*ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['9611',
 '403879',
 '442858',
 '403811',
 '403869',
 '475738',
 '403802',
 '403885',
 '442942',
 '804479',
 '403424',
 '403418',
 '403980',
 '610255',
 '403955',
 '403559',
 '477032',
 '404306',
 '403998',
 '403649',
 '403752']

In [22]:
number_of_genes = int(input("Número de genes a obter (máx: 20): "))
print(number_of_genes)
genes = id_gene[0:number_of_genes]
print(genes)

6
['9611', '403879', '442858', '403811', '403869', '475738']


In [7]:
#BUSCAR LINKS DOS GENES

In [23]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/9611',
 'https://www.ncbi.nlm.nih.gov/nuccore/403879',
 'https://www.ncbi.nlm.nih.gov/nuccore/442858',
 'https://www.ncbi.nlm.nih.gov/nuccore/403811',
 'https://www.ncbi.nlm.nih.gov/nuccore/403869',
 'https://www.ncbi.nlm.nih.gov/nuccore/475738']

In [24]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [25]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
#r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')
	print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       X00593                  2029 bp    DNA     linear   INV 18-APR-2005
DEFINITION  Lytechinus pictus H3 and H4 gene.
ACCESSION   X00593
VERSION     X00593.1
KEYWORDS    histone; histone H3; histone H4.
SOURCE      Lytechinus pictus (painted urchin)
  ORGANISM  Lytechinus pictus
            Eukaryota; Metazoa; Echinodermata; Eleutherozoa; Echinozoa;
            Echinoidea; Euechinoidea; Echinacea; Temnopleuroida;
            Toxopneustidae; Lytechinus.
REFERENCE   1  (bases 1 to 2029)
  AUTHORS   Roberts,S.B., Weisser,K.E. and Childs,G.
  TITLE     Sequence comparisons of non-allelic late histone genes and their
            early stage counterparts. Evidence for gene conversion within the
            sea urchin late stage gene family
  JOURNAL   J. Mol. Biol. 174 (4), 647-662 (1984)
   PUBMED   6587117
FEATURES             Location/Qualifiers
     source          1..2029
                     /organism="Lytechinus pictus"
                     /mol_type="genomic DNA"
            

LINK NCBI

In [26]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista



['https://www.ncbi.nlm.nih.gov/nuccore/X00593',
 'https://www.ncbi.nlm.nih.gov/nuccore/U01502',
 'https://www.ncbi.nlm.nih.gov/nuccore/U01468',
 'https://www.ncbi.nlm.nih.gov/nuccore/U01497']

ACCESSION

In [27]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['X00593 ', 'U01502 ', 'U01468 ', 'U01497 ']

VERSION

In [46]:
version_list = []

existe = re.findall(r'VERSION.*?(?=KEYWORDS)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for vers in existe:
        m = re.match( r'VERSION\s+(.+)', vers, re.DOTALL )
        version_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
version_list

['X00593.1 ', 'U01502.1 ', 'U01468 ', 'U01497 ']

DEFINITION

In [28]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['Lytechinus pictus H3 and H4 gene. ',
 'Human immunodeficiency virus type 1 clone 820-99 tat gene, partial cds. ',
 'Human immunodeficiency virus type 1 clone 810-121 tat gene, partial cds. ',
 'Human immunodeficiency virus type 1 clone 820-64 tat gene, partial cds. ']

LOCUS

In [29]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['X00593 2029 bp DNA linear INV 18-APR-2005 ',
 'HIV1U01502 312 bp RNA linear VRL 30-DEC-1994 ',
 'U01468 285 bp RNA linear VRL 29-DEC-1994 ',
 'U01497 312 bp RNA linear VRL 29-DEC-1994 ']

LOCUS NAME

In [30]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list

['X00593', 'HIV1U01502', 'U01468', 'U01497']

NUMBER OF BP

In [31]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['2029 bp', '312 bp', '285 bp', '312 bp']

MOLECULE TYPE

In [32]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['DNA linear', 'RNA linear', 'RNA linear', 'RNA linear']

GENBANK DIVISION

In [33]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['INV', 'VRL', 'VRL', 'VRL']

MODIFICATION DATE

In [34]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['18-APR-2005', '30-DEC-1994', '29-DEC-1994', '29-DEC-1994']

SOURCE

In [35]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Lytechinus pictus (painted urchin) ',
 'Human immunodeficiency virus 1 (HIV-1) ',
 'Unknown. ',
 'Unknown. ']

ORGANISM

In [37]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Lytechinus pictus', 'Human immunodeficiency virus 1', 'Unknown.', 'Unknown.']

FEATURES

In [ ]:
misc = []
existe = re.findall(r'FEATURES\s+.*?(?=ORIGIN)', locus, re.DOTALL)
#DOTALL ignora as linhas, pois as features ocupam várias linhas
if existe:
    for feat in existe:
        m = re.match( r'FEATURES\s+(.+)',feat, re.DOTALL )
        misc.append(re.sub(r'/',  ' ', m.group(1) ) )
    for c in misc:
        print(c, "\n")

FEATURES - SOURCE

In [ ]:

existe = re.findall(r'source\s+[1-9\.]*', locus, re.DOTALL)
if existe:
    for fsource in existe:
        print( re.sub(r'source\s+', ' ', fsource) )


FEATURES - EXONS

In [ ]:
existe = re.findall(r'exon\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fexons in existe:
        print( re.sub(r'exon\s+', ' ', fexons) )

FEATURES - INTRON

In [ ]:
existe = re.findall(r'intron\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fintrons in existe:
        print( re.sub(r'intron\s+', ' ', fintrons) )

CDS - JOIN

In [ ]:
existe = re.findall(r'CDS\s+?(join)[1-9\.\,\(\)]*', locus, re.DOTALL)
#(join) procura a palavra join 1x
#procura nrs de 1-9, ponto, virgula e parentises zero ou mais vezes
print(existe[0])
if existe:
    for cds in existe:
        print( re.sub(r'CDS\s+', ' ', cds) )

SEQUENCE/ORIGIN

In [38]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['gtacataattataacacataaataagagacattcaaattgtttctttcttgaatatttggtggctctgaaaagagccggtttgttgatgggtttgaagagatgaacttcttaacctccgaatccgtagagggtacgtccctgcctcttcagggcgtagacgacgtccatggcggtgacggtcttcctcttagcgtgctcgcagtaggtgacggcatcacggataacattctcaaggaagaccttgaggacaccacgggtctcttcgtagatgagaccactgatacgcttgacaccaccacggcgggcaagacgacgaatggctggcttggtgattccttggatgttatcacgcaagaccttgcgatgacgcttggcgccaccctttccgagtcctttacctcctttgccacgtccagacatgattagatttatttggtatcgacgagtcgaattgacgtgataggatttcgaccggtatatatacccctattccggacgtcgaaggaatagtaatgctctctggatcaatgagctctctctgttcatcaaacgtagagggcgatattttaaaacatgttatttgcatccgtgtatcaacggtatcttgaagtagcaatgtagtactaaaatacatgagttataatgctcgaaatgttagaattagcaaaattgatgtaaacgcacagtacaaatgacagttttgcattgttttatactaacattcttatcatttgaattctgaattctaaatatttagtaaaatatgtctgtgtatcactcaaaaagaagaccgggatatcctcacgagcaactgggatatttgcataaattttcgtaatgatagatacattcttttcttagttttcgcagttttttttttatttttttattcaacttcgattcgcttaaatcacgtgtcaatatacacattctctatcgtttaaaatatcaaaatatgaaaccaggcaaaaattatagcaatattattatttatatg

In [39]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[587, 461, 569, 412], [110, 63, 74, 65], [99, 59, 70, 57], [110, 63, 74, 65]]

In [40]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[2029, 312, 285, 312]

JOURNAL

In [41]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['J. Mol. Biol. 174 (4), 647-662 (1984) ',
 'AIDS 8 (7), 901-909 (1994) ',
 'Submitted (09-SEP-1993) Alan Mayer, Irwin Memorial Blood Centers, Scientific Service, 270 Masonic Av, San Francisco, CA 94118 USA ',
 'AIDS 8, 901-909 (1994) ',
 'Submitted (09-SEP-1993) Mayer A., Irwin Memorial Blood Centers, Scientific Service, 270 Masonic Av, San Francisco, CA 94118 USA COMMENT [WARNING] On Dec 30, 1994 this sequence was replaced by U01468.1. ',
 'AIDS 8, 901-909 (1994) ',
 'Submitted (09-SEP-1993) Mayer A., Irwin Memorial Blood Centers, Scientific Service, 270 Masonic Av, San Francisco, CA 94118 USA COMMENT [WARNING] On Dec 30, 1994 this sequence was replaced by U01497.1. ']

AUTHORS

In [42]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['Roberts,S.B., Weisser,K.E. and Childs,G. ',
 'Sabino,E., Pan,L.Z., Cheng-Mayer,C. and Mayer,A. ',
 'Mayer,A. ',
 'Sabino,E., Pan,L.Z., Cheng-Mayer,C. and Mayer,A. ',
 'Mayer,A. ',
 'Sabino,E., Pan,L.Z., Cheng-Mayer,C. and Mayer,A. ',
 'Mayer,A. ']

TITLE

In [43]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Sequence comparisons of non-allelic late histone genes and their early stage counterparts. Evidence for gene conversion within the sea urchin late stage gene family ',
 'Comparison of in vivo plasma and peripheral blood mononuclear cell HIV-1 quasi-species to short-term tissue culture isolates: an analysis of tat and C2-V3 env regions ',
 'Direct Submission ',
 'Comparison of in vivo plasma and peripheral blood mononuclear cell HIV-1 quasi-species to short-term tissue culture isolates: an analysis of tat and C2-V3 env regions ',
 'Direct Submission ',
 'Comparison of in vivo plasma and peripheral blood mononuclear cell HIV-1 quasi-species to short-term tissue culture isolates: an analysis of tat and C2-V3 env regions ',
 'Direct Submission ']

PUBMED

In [118]:
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', locus, re.DOTALL)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)', pubm, re.DOTALL )
        print( re.sub(r'\s+', ' ', m.group(1) ) )

6587117 FEATURES Location/Qualifiers source 1..2029 /organism="Lytechinus pictus" /mol_type="genomic DNA" /db_xref="taxon:7653" CDS complement(110..421) /note="unnamed protein product; H4" /codon_start=1 /protein_id="CAA25241.1" /db_xref="GOA:P62782" /db_xref="InterPro:IPR001951" /db_xref="InterPro:IPR007125" /db_xref="InterPro:IPR009072" /db_xref="InterPro:IPR019809" /db_xref="UniProtKB/Swiss-Prot:P62782" /translation="MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGV KRISGLIYEETRGVLKVFLENVIRDAVTYCEHAKRKTVTAMDVVYALKRQGRTLYGFG G" CDS 1478..1888 /note="unnamed protein product; H3" /codon_start=1 /protein_id="CAA25242.1" /db_xref="GOA:P69072" /db_xref="InterPro:IPR000164" /db_xref="InterPro:IPR007125" /db_xref="InterPro:IPR009072" /db_xref="UniProtKB/Swiss-Prot:P69072" /translation="MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRYRP GTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTELRFQSSAVMALQEASEAYLV GLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA" ORIGIN 1 gtacataatt ataacacata aataagagac attcaaattg tttctttctt gaatatttg

In [122]:
import requests
from bs4 import BeautifulSoup

def url_get_id_pm(item):
    url_id_pm = []
    for id_pm in genes:
        url = f"https://pubmed.ncbi.nlm.nih.gov/{id_pm}/"
        url_id_pm.append(url)
    return url_id_pm

pm = ""
soup = []
content_pm = []
for url in url_get_id_pm(genes):
	r2 = requests.get(url)
	pm += r2.content.decode('utf-8')
	soup.append(BeautifulSoup(r2.content, 'html.parser'))
	content_pm.append(r2.content.decode('utf-8'))

print(soup)
print(content_pm)




[<!DOCTYPE html>

<html lang="en">
<head itemscope="" itemtype="http://schema.org/WebPage" prefix="og: http://ogp.me/ns#">
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- Mobile properties -->
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://cdn.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.google-analytics.com" rel="preconnect"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.e3e95ed4a5de.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.452c70ce66f7.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.dc65705d

AFFILIATION

In [119]:
import requests
r = requests.get('https://pubmed.ncbi.nlm.nih.gov/2549509')
pub = r.content.decode('utf-8')
print(pub)

<!DOCTYPE html>




<html lang="en" >
<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <!-- Mobile properties -->
    <meta name="HandheldFriendly" content="True">
    <meta name="MobileOptimized" content="320">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

  
  
  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.google-analytics.com">

  
  
    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.e3e95ed4a5de.css" type="text/css">
  

  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.452c70ce66f7.css" type="text/css">

  
    
  

  
    <link rel="stylesheet" href="https://cdn.n

In [128]:
import re
for x in content_pm:
    existe = re.search(r'affiliation-1\s*.*\.', x)
#\s* zero ou mais espaços
#.* zero ou mais caracteres
#\. caractere ponto final -> procura affiliation até encontrar ponto final
    print(x)

    if existe:
        m = re.findall(r'>\s.*', existe[0] )
        #o findall retorna uma lista, por isso temos que selecionar m[0] para fazer o replace
        m = m[0].replace("> ",'')
        print(m)

    else:
        print( "Padrão não encontrado" )


<!DOCTYPE html>




<html lang="en" >
<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <!-- Mobile properties -->
    <meta name="HandheldFriendly" content="True">
    <meta name="MobileOptimized" content="320">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

  
  
  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.google-analytics.com">

  
  
    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.e3e95ed4a5de.css" type="text/css">
  

  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.452c70ce66f7.css" type="text/css">

  
    
  

  
    <link rel="stylesheet" href="https://cdn.n

AUTHOR NAME

In [ ]:
existe = re.search("citation_authors.*;", pub)
if existe:
    m = re.findall(r'=".*', existe[0] )
    #procura =" seguido de um ou mais caracteres
    m = m[0].replace('=\"','')
    #\" para indicar que queremos os caractere " (e não abrir aspas)
    print(m)

CONNECTION TO DATABASE

In [116]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `LOCUS` (
        `Id_locus` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Accession` VARCHAR(20) NOT NULL,
        `Sequence_length` INT NULL,
        `Molecule_type` VARCHAR(45) NULL,
        `GB_division` VARCHAR(10) NULL,
        `Source` VARCHAR(45) NULL,
        `Organism` VARCHAR(45) NULL,
        `Locus_name` VARCHAR(45) NULL
        );
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES"
    for index in range(len(loc_name_list)):
        if index == len(loc_name_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

4 record inserted.


In [117]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `GENBANK` (
        `Id_GB` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `GB_id` VARCHAR(45) NOT NULL,
        `Definition` VARCHAR(200) NULL,
        `Modification_date` VARCHAR(11) NULL,
        `Version` VARCHAR(10) NOT NULL UNIQUE,
        `LOCUS_Id_locus` INT NOT NULL,
        `ORIGIN_Id_origin` INT NULL
    )
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES"
    

    for index in range(len(version_list)):
        sql_insert_origin = "INSERT INTO ORIGIN (Length, Adenine, Citosine, Timine , Guanine) VALUES" 
        sql_insert_origin += f" (\"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
        Cursor.execute(sql_insert_origin)
        # print(Cursor.lastrowid)  #buscar o id do origin inserido
        id_inserted_origin = Cursor.lastrowid


        select_locus = f"SELECT Id_locus from LOCUS where Locus_name = '{loc_name_list[index]}'"
        Cursor.execute(select_locus)
        myresult = Cursor.fetchall()
        for res, in myresult:
            id_locus = res
        # print(id_locus)
        
        if index == len(version_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

4 record inserted.
